<a href="https://colab.research.google.com/github/dangdang2222/LG_Aimers/blob/main/optuna_kfold_y_06_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import random
import os
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,13): # ignore 'ID'
        rmse = mean_squared_error(np.array(gt)[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(np.array(gt)[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [4]:
def lg_nrmse_one(gt, preds):
    # Y_08의 경우는 가중치 1.2 부여. 8 이후부터는 모두 가중치 1.0
    all_nrmse = []
    rmse = mean_squared_error(np.array(gt), preds, squared=False)
    nrmse = rmse/np.mean(np.abs(np.array(gt)))
    all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse)
    return score

#얘를 쓸것!!!

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/LG_Aimer/train.csv').drop(columns="ID")
train_x = train_df.filter(regex='X') # Input : X Featrue
train_x=train_x.drop(columns=['X_04','X_23','X_47','X_48'])
train_y = train_df.filter(regex='Y') # Output : Y Feature

##Y_03


In [6]:
train_y06 = train_y['Y_06']
train_y06
train_y06=pd.DataFrame(train_y06, index= train_y.index)
train_y06

,Y_06
0,16.083
1,16.736
2,17.080
3,17.143
4,17.569
...,...
39602,16.582
39603,15.659
39604,16.823
39605,15.757


#클리핑

In [7]:
from sklearn.model_selection import train_test_split

# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x.quantile(0.01)
p99 = train_x.quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x = train_x.clip(p01, p99, axis=1)

In [8]:
# 커넥터 핀 휨 
train_x['X__21'] = (train_x['X_24'] - train_x['X_12'])
train_x['X__22'] = (train_x['X_25'] - train_x['X_12'])
train_x['X__23'] = (train_x['X_26'] - train_x['X_12'])
train_x['X__24'] = (train_x['X_27'] - train_x['X_12'])
train_x['X__25'] = (train_x['X_28'] - train_x['X_12'])
train_x['X__26'] = (train_x['X_29'] - train_x['X_12'])
#이후, RFE 상에서 혹여나 새로운 변수를 만드는데 기여한 기존 변수들(12, 24,25,26,27,28,29)가 important하게 잡힐 수 있으므로 일단은 놔둡니다.
#또한, 현재 train_x 상에서는 통과여부 변수들은 모두 drop된 상태입니다.

In [9]:
new_col = ['X_49', 'X_09', 'X_14', 'X_05', 'X_15', 'X_07', 'X_03', 'X_02', 'X_38', 'X_16', 'X_01', 'X_50', 'X_56', 'X_08', 'X_44', 'X_22', 'X_41', 'X_21','X_42']
print(len(new_col))
train_set = train_x[new_col]
train_set

19


,X_49,X_09,X_14,X_05,X_15,X_07,X_03,X_02,X_38,X_16,X_01,X_50,X_56,X_08,X_44,X_22,X_41,X_21,X_42
0,9706.030,245.71,13.34,101.892,13.37,29.45,67.47,103.320,-16.41,13.40,70.544,137.043591,125.028256,62.38,21.09,3.13,21.20,3.06,20.9900
1,10423.430,233.61,13.33,101.944,13.33,28.73,65.17,103.321,-16.06,13.46,69.524,133.736691,124.877308,61.23,21.13,3.20,21.16,2.98,21.0300
2,10948.530,272.20,13.36,103.153,13.36,28.81,64.07,103.320,-16.16,13.39,72.583,132.805112,122.238232,105.77,21.12,3.12,21.13,3.01,21.0300
3,15007.030,255.36,13.30,101.971,13.33,28.92,67.57,103.320,-16.05,13.39,71.563,134.138760,134.875225,115.21,21.09,3.08,21.18,3.02,20.9800
4,11051.030,241.46,13.35,101.981,13.34,29.68,63.57,103.320,-16.25,13.41,69.524,142.728970,123.272762,103.38,21.10,3.12,21.16,3.00,20.9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,56583.294,298.05,13.37,103.150,13.38,30.20,62.27,103.320,-16.09,13.51,66.465,129.965741,129.029812,77.83,21.19,3.13,21.17,3.06,21.0800
39603,56583.294,270.67,13.36,102.021,13.36,29.21,62.77,103.321,-15.70,13.47,66.465,127.633885,122.811413,102.25,21.19,3.06,21.16,3.05,21.0900
39604,9092.372,198.07,13.38,103.144,13.39,29.96,64.67,103.320,-16.12,13.47,68.504,132.501286,119.166699,102.61,21.19,3.12,21.17,3.07,21.0900
39605,56583.294,275.52,13.36,102.025,13.38,30.30,63.67,103.320,-16.36,13.45,66.465,128.189679,124.525929,112.60,21.13,3.09,21.15,3.15,21.0500


In [10]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 4.0 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 59.8 MB/s 
     |████████████████████████████████| 78 kB 8.8 MB/s 
     |████████████████████████████████| 147 kB 71.9 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 112 kB 72.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=4c8c7431f3e7ef7977741a8e889d080a514a3530ec6d67dc9134d93fcb518c55
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [11]:
import optuna
import time
from optuna import Trial, visualization
from sklearn.model_selection import KFold
import xgboost as xgb


#XGB 하이퍼 파라미터들 값 지정
def objectiveXGB(trial: Trial, X, y):  # X : train_set, y = train_y02

    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 2000, 4000,500),
        'max_depth':trial.suggest_int('max_depth', 8, 13),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 20),
        'gamma':1,
        'learning_rate': trial.suggest_discrete_uniform('learning_rate',0.004,0.01,0.0005),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.05),
        'nthread' : -1,
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.5,0.55,0.6,0.65,0.7,0.75])
    }

    kf = KFold(shuffle=True, n_splits=5, random_state=42)
    loss = []
    i = 0

    # 데이터 나누기
    X, y = X.values, y.values
    for (train_index, test_index) in kf.split(X):
        since = time.time()
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # 설정된 파라미터 적용
        model = xgb.XGBRegressor(**param)

        # 학습하기
        xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])

        # 평가 및 기록
        elapsed = time.time() - since
        score = lg_nrmse_one(y_test, xgb_model.predict(X_test))
        print("Fold {} | NRMSE : {:.6f}, Elapsed Time: {:.4f}".format(i, score, elapsed))
        loss.append(score)
        i += 1

    print("Best: {:.6f}, Avg:{:.6f}".format(min(loss), np.mean(loss)))

    return np.mean(loss)

0.12298070045135383

In [ ]:
from optuna.samplers import TPESampler

# MAE가 최소가 되는 방향으로 학습을 진행
# TPESampler : Sampler using TPE (Tree-structured Parzen Estimator) algorithm.

# x_train, x_valid, y_train, y_valid = train_test_split(train_x_xgb, train_y01, test_size=0.3) 
study = optuna.create_study(direction='minimize', sampler=TPESampler())

# n_trials 지정해주지 않으면, 무한 반복
study.optimize(lambda trial : objectiveXGB(trial, train_set, train_y06), n_trials = 100)

print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
print('study.best_params:', study.best_trial.value)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('study.best_params:', study.best_params)

[I 2022-08-25 15:12:12,414] A new study created in memory with name: no-name-c3cb9a9f-cf85-4b1a-b427-981b31933206


[15:12:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.113328, Elapsed Time: 254.8627
[15:16:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.119978, Elapsed Time: 256.0867
[15:20:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.129012, Elapsed Time: 253.7995
[15:25:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.121124, Elapsed Time: 255.4101
[15:29:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 15:33:47,356] Trial 0 finished with value: 0.12211506393147627 and parameters: {'n_estimators': 3500, 'max_depth': 11, 'min_child_weight': 4, 'learning_rate': 0.006, 'colsample_bytree': 0.75, 'lambda': 2.51821057837246, 'alpha': 0.010659780128907559, 'subsample': 0.75}. Best is trial 0 with value: 0.12211506393147627.


Fold 4 | NRMSE : 0.127134, Elapsed Time: 256.2668
Best: 0.113328, Avg:0.122115
[15:33:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.113521, Elapsed Time: 168.8763
[15:36:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.122811, Elapsed Time: 168.8805
[15:39:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.129845, Elapsed Time: 168.8080
[15:42:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.121249, Elapsed Time: 167.5546
[15:45:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 15:48:03,958] Trial 1 finished with value: 0.12320975412322972 and parameters: {'n_estimators': 2000, 'max_depth': 13, 'min_child_weight': 1, 'learning_rate': 0.005, 'colsample_bytree': 0.75, 'lambda': 3.859930976045342, 'alpha': 0.005624550443401086, 'subsample': 0.5}. Best is trial 0 with value: 0.12211506393147627.


...

Fold 4 | NRMSE : 0.128825, Elapsed Time: 278.5658
Best: 0.115653, Avg:0.124255
[17:48:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.115544, Elapsed Time: 219.9344
[17:51:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.121732, Elapsed Time: 223.5202
[17:55:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.130826, Elapsed Time: 219.5856
[17:59:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.123417, Elapsed Time: 220.6375
[18:03:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 18:06:54,352] Trial 8 finished with value: 0.12421274601144994 and parameters: {'n_estimators': 3500, 'max_depth': 12, 'min_child_weight': 9, 'learning_rate': 0.008, 'colsample_bytree': 0.55, 'lambda': 0.002656016354049105, 'alpha': 0.23643451962101716, 'subsample': 0.6}. Best is trial 0 with value: 0.12211506393147627.


Fold 4 | NRMSE : 0.129546, Elapsed Time: 222.9012
Best: 0.115544, Avg:0.124213
[18:06:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.113627, Elapsed Time: 179.8870
[18:09:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.121519, Elapsed Time: 180.4552
[18:13:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.130267, Elapsed Time: 179.6345
[18:16:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.121809, Elapsed Time: 185.1208
[18:19:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 18:22:16,680] Trial 9 finished with value: 0.12318994323922172 and parameters: {'n_estimators': 3000, 'max_depth': 11, 'min_child_weight': 5, 'learning_rate': 0.0055, 'colsample_bytree': 0.6, 'lambda': 1.8256147660457342, 'alpha': 0.024622207569360576, 'subsample': 0.75}. Best is trial 0 with value: 0.12211506393147627.


Fold 4 | NRMSE : 0.128728, Elapsed Time: 181.7788
Best: 0.113627, Avg:0.123190
[18:22:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.114306, Elapsed Time: 128.4724
[18:24:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
